In [6]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
#output display setting
torch.set_grad_enabled(True)

In [2]:
print(torch.__version__)
print(torchvision.__version__)

1.1.0
0.3.0


In [3]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

## 建立神经网络

In [4]:
class Network(nn.Module):
#有继承的类需要把父类写在括号里
    def __init__(self):
        super(Network, self).__init__()
#规定操作，第三行要写父类的构造器，格式为super(子类，self).__init__（）括号内可
#能有父亲类的构造参数。
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
#in_channels=1因为输入的图片为灰阶图片，此超参数由数据影响
#类的属性可以是其他类
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2= nn.Linear(in_features=120, out_features=60)
        
        self.out = nn.Linear(in_features=60, out_features=10)
#out_features=10由数据决定，分成十类。

#一般来说一个层的输出数目就是下一个层的输入数目。


        
#Network类包含属性：conv1，conv2，fc1，fc2和out一共5个属性，每个属性的值由nn
#下面的一个类定义而成（类的属性可以是其他类），类位置在nn/modole/conv中并且加
#入了适当的构造参数。其构造参数即为神经网络的“超参数”。超参数由设计者根据经验给出。
#
#Con2d为图片卷积层：包含三个参数：kernel_size，in_channels，out_features。
#kernel_size：定义了卷积核的大小
#in_channels：定义了输入的通道数
#out_channels：定义了卷积核的数量，每个卷积核都能生成一个卷积图片（feature_maps）
# out_features：全连阶层的输出，设计者决定。
#一般来说附加的卷积层会提升输出的通道，线性层会一层一层收缩。
#

    def forward(self, t): 
        #(1) input layer
        t=t
        
        #(2) hidden conv layer
        t=self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        #relu和pooling不包含权重参数，只是操作，所以直接从torch.nn.functional引用
        #大小为2的池化层会将每一个2x2的区域找出最大值并且返回，输出图片是之前的一半
        
        #(3) hidden conv layer
        t=self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        #(4) hidden linear layer 
        t = t.reshape(-1,12*4*4)
        t = self.fc1(t)
        t = F.relu(t)
        
        #(5) hidden linear layer 
        t = self.fc2(t)
        t = F.relu(t)
        
        #(6) output layer
        t = self.out(t)
        #t = F.softmax(t, dim=1)
        
        #hidden layers多用ReLU激活函数，单一预测的输出层多用softmax激活函数
        
        return t

In [16]:
net_fast = nn.Sequential(
        nn.Conv2d(1,6,5),
        nn.ReLU(),
        nn.MaxPool2d(2, stride=2),
        nn.Conv2d(6,12,2),
        nn.ReLU(),
        nn.MaxPool2d(2, stride=2),
    #但是Reshape无法做？？
    
)

In [6]:
train_set = torchvision.datasets.FashionMNIST(
        root='./data/FashionMNIST',
        train=True,
        download=True,
        transform=transforms.Compose([transforms.ToTensor()])
        )

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
batch=next(iter(train_loader))
images, labels = batch
images.shape

torch.Size([100, 1, 28, 28])

In [7]:
net = Network()
net

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)

## 损失函数计算

In [8]:
preds = net(images)
loss = F.cross_entropy(preds, labels)
#loss函数会追踪计算图，用来做反向传播
loss.item()

2.3089704513549805

In [9]:
print(preds.shape)
print(labels.shape)
#怎么比较的 交叉entropy，不同维度

torch.Size([100, 10])
torch.Size([100])


## 计算梯度

In [10]:
print(net.conv1.weight.grad)

None


In [11]:
retain_graph=True

In [12]:
loss.backward()
#根据loss的计算图，将每个参数的梯度计算出来，位置对应

In [15]:
print(net.conv1.weight.grad.shape)
print(net.conv1.weight.shape)

#梯度张量的形状和对应层权重一样

torch.Size([6, 1, 5, 5])
torch.Size([6, 1, 5, 5])


## 优化

In [25]:
optimizer = optim.Adam(net.parameters(), lr=0.01)
#将神经网络的参数导入优化器，包括当前权重，当前梯度

In [18]:
loss.item()

2.3089704513549805

In [23]:
get_num_correct(preds, labels)

12

In [26]:
optimizer.step()
#优化器的step方法，做一次优化

In [27]:
preds = net(images)
loss = F.cross_entropy(preds, labels)

In [28]:
loss.item()

2.257197380065918

In [30]:
get_num_correct(preds, labels)

27

In [33]:
F.softmax(preds, dim=1)

tensor([[0.1401, 0.0936, 0.0893, 0.1422, 0.0614, 0.1250, 0.0734, 0.0702, 0.0576, 0.1473],
        [0.1404, 0.0962, 0.0879, 0.1463, 0.0599, 0.1278, 0.0751, 0.0677, 0.0564, 0.1421],
        [0.1254, 0.1013, 0.0952, 0.1249, 0.0764, 0.1160, 0.0918, 0.0766, 0.0764, 0.1159],
        [0.1298, 0.1005, 0.0934, 0.1318, 0.0711, 0.1199, 0.0868, 0.0735, 0.0701, 0.1232],
        [0.1407, 0.0968, 0.0847, 0.1508, 0.0599, 0.1274, 0.0742, 0.0671, 0.0562, 0.1421],
        [0.1389, 0.0953, 0.0901, 0.1417, 0.0605, 0.1278, 0.0761, 0.0697, 0.0576, 0.1424],
        [0.1308, 0.0959, 0.0946, 0.1318, 0.0665, 0.1243, 0.0832, 0.0761, 0.0650, 0.1318],
        [0.1426, 0.0931, 0.0882, 0.1456, 0.0573, 0.1280, 0.0725, 0.0682, 0.0535, 0.1510],
        [0.1257, 0.0993, 0.0970, 0.1244, 0.0749, 0.1190, 0.0897, 0.0776, 0.0754, 0.1169],
        [0.1321, 0.0954, 0.0925, 0.1357, 0.0659, 0.1279, 0.0813, 0.0725, 0.0641, 0.1326],
        [0.1355, 0.0985, 0.0906, 0.1395, 0.0653, 0.1238, 0.0806, 0.0698, 0.0628, 0.1334],
        [0